# Proyecto: Selección de región óptima para perforación de pozos petroleros
Este proyecto evalúa tres regiones con datos sintéticos para determinar dónde perforar 200 nuevos pozos de petróleo. Se utiliza regresión lineal para predecir reservas, se estima el beneficio y se analiza el riesgo con bootstrapping.

In [23]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns




## 1. Carga y exploración de los datos

In [24]:
data_0 = pd.read_csv('/Users/juanvielmapereyra/NOTEBOOKS/SPRINT_11/geo_data_0.csv')
data_1 = pd.read_csv('/Users/juanvielmapereyra/NOTEBOOKS/SPRINT_11/geo_data_1.csv')
data_2 = pd.read_csv('/Users/juanvielmapereyra/NOTEBOOKS/SPRINT_11/geo_data_2.csv')

for i, data in enumerate([data_0, data_1, data_2]):
    print(f"Región {i}:")
    print(data.info())
    print(data.describe())
    print()


Región 0:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331   

## 2. Entrenamiento y evaluación de modelo de regresión lineal

In [25]:
def train_region_model(data):
    features = data.drop(['id', 'product'], axis=1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=42)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    from math import sqrt
    rmse = sqrt(mean_squared_error(target_valid, predictions))

    return model, features_valid, target_valid, predictions, rmse

results = {}
for i, data in enumerate([data_0, data_1, data_2]):
    model, f_val, t_val, pred, rmse = train_region_model(data)
    results[f'Región {i}'] = {
        'model': model,
        'features_valid': f_val,
        'target_valid': t_val,
        'predictions': pred,
        'rmse': rmse
    }
    print(f"Región {i} - RMSE: {rmse:.2f}, Volumen predicho promedio: {np.mean(pred):.2f}, Real: {t_val.mean():.2f}")


Región 0 - RMSE: 37.76, Volumen predicho promedio: 92.40, Real: 92.33
Región 1 - RMSE: 0.89, Volumen predicho promedio: 68.71, Real: 68.73
Región 2 - RMSE: 40.15, Volumen predicho promedio: 94.77, Real: 95.15


## 3. Cálculo de beneficios estimados

In [26]:
NUM_WELLS = 200
BUDGET = 100_000_000
REVENUE_PER_UNIT = 4_500
BREAK_EVEN_PRODUCTION = BUDGET / (NUM_WELLS * REVENUE_PER_UNIT)

def calculate_profit(target, predictions):
    selected_indices = predictions.argsort()[-NUM_WELLS:]
    return target.iloc[selected_indices].sum() * REVENUE_PER_UNIT - BUDGET

profits = {}
for i in range(3):
    region = f'Región {i}'
    profit = calculate_profit(results[region]['target_valid'], pd.Series(results[region]['predictions']))
    profits[region] = profit
    print(f"{region}: Ganancia estimada ${profit:,.2f}")


Región 0: Ganancia estimada $33,591,411.14
Región 1: Ganancia estimada $24,150,866.97
Región 2: Ganancia estimada $25,985,717.59


## 4. Análisis de riesgos con bootstrapping

In [27]:
def bootstrap_profit(target, predictions, n_iterations=1000):
    state = np.random.RandomState(42)
    values = []
    for _ in range(n_iterations):
        sample = state.choice(predictions.index, size=len(predictions), replace=True)
        pred_sample = predictions[sample]
        target_sample = target.iloc[sample]
        profit = calculate_profit(target_sample, pred_sample)
        values.append(profit)
    values = pd.Series(values)
    return values.mean(), values.quantile(0.025), values.quantile(0.975), (values < 0).mean()

for i in range(3):
    region = f'Región {i}'
    pred = pd.Series(results[region]['predictions']).reset_index(drop=True)
    target = results[region]['target_valid'].reset_index(drop=True)
    mean_, low, high, risk = bootstrap_profit(target, pred)
    print(f"{region}: Prom. beneficio ${mean_:,.2f}, IC 95%: [{low:,.2f}, {high:,.2f}], Riesgo pérdida: {risk*100:.2f}%")


Región 0: Prom. beneficio $33,719,318.41, IC 95%: [30,437,973.79, 37,376,105.23], Riesgo pérdida: 0.00%
Región 1: Prom. beneficio $24,150,866.97, IC 95%: [24,150,866.97, 24,150,866.97], Riesgo pérdida: 0.00%
Región 2: Prom. beneficio $25,601,869.13, IC 95%: [21,666,374.07, 29,186,711.85], Riesgo pérdida: 0.00%


## 5. Conclusiones finales
Con base en la ganancia estimada y el análisis de riesgo con bootstrapping, se recomienda la región que maximiza el beneficio y minimiza el riesgo.
Esto asegura una mejor decisión para la inversión de 100 millones de USD en 200 pozos.

Región 0 es la más rentable en términos absolutos.

Las tres regiones cumplen el requisito mínimo de producir más de 111.1 unidades por pozo, pero Región 0 sobresale en volumen y ganancia.